<a id="top"></a>
Following Workflow

### [File Storage](#File_Storage)
* [Create](#File_Storage_Create) an **S3 Bucket**
* [Load](#File_Storage_Load) csv file and script to execute
* [Delete](#File_Storage_Delete) object and content

### [Execute Code](#Execute_Code)
#### [EC2](#Execute_Code_EC2) Miniconda Image
* [Create](#Execute_Code_EC2_Create) the EC2 Instance
* [Mount](#Execute_Code_EC2_Mount) S3 Storage
* Determine mechanism to execute script
* Shut down server

#### SageView Instance
* Create SageView
* Mount S3 Storage
* Execute Script

### Requirements
pycloud environment
```
conda env create -f pycloud.env.ymp --force
```

From Command line execute `aws configure` in order to setup your AWS Access Key and AWS Secret Key

Local Test Code is located under `example_src`

[Go back to top](#top)
## File Storage<a id="File_Storage"></a>
Code to create an S3 Storage
https://realpython.com/python-boto3-aws-s3/#creating-a-bucket

### Create S3 Object<a id="File_Storage_Create"></a>

In [31]:
import uuid
def create_bucket_name(bucket_prefix):
    # The generated bucket name must be between 3 and 63 chars long
    return ''.join([bucket_prefix, str(uuid.uuid4())])

def create_bucket(bucket_prefix, s3_connection):
    session = boto3.session.Session()
    current_region = session.region_name
    # Create Configurations
    configs = {}
    if current_region != "us-east-1":
        config["LocationConstraint"] = current_region

    bucket_name = create_bucket_name(bucket_prefix)
    
    if current_region == "us-east-1":
        bucket_response = s3_connection.create_bucket(
            Bucket=bucket_name)
    else:
        bucket_response = s3_connection.create_bucket(
            Bucket=bucket_name,
            CreateBucketConfiguration={"LocationConstraint":current_region})
        
    print(bucket_name, current_region)
    return bucket_name, bucket_response

In [129]:
import boto3

s3_resource = boto3.resource('s3')
bucket_name, first_response = create_bucket(
    bucket_prefix='iris-train', 
    s3_connection=s3_resource.meta.client)

iris-trainc0e3588c-d9bb-4699-821c-1883670ace42 us-east-1


### Load Files into S3 Object<a id="File_Storage_Load"></a>
There is no native folder sync within Python SDK, so using aws command to solve for this problem!

In [132]:
import os
import subprocess
path_to_src = "../example_src"
path_to_s3 = "s3://" + bucket_name


result = subprocess.run(["aws","s3", "sync" ,path_to_src, path_to_s3,"--acl","private"], stdout=subprocess.PIPE)

[ print( l ) for l in result.stdout.decode('utf-8').split('\n') ]


upload: ../example_src/requirements.yml to s3://iris-trainc0e3588c-d9bb-4699-821c-1883670ace42/requirements.yml



[None, None]

### Delete S3 Object and Content<a id="File_Storage_Delete"></a>
Remove all resources and delete bucket!<br>
**This does not back-up anything!**

In [128]:
def delete_all_objects(bucket_name):
    res = []
    bucket=s3_resource.Bucket(bucket_name)
    for obj_version in bucket.object_versions.all():
        res.append({'Key': obj_version.object_key,
                    'VersionId': obj_version.id})
    print(res)
    bucket.delete_objects(Delete={'Objects': res})


delete_all_objects("iris-trained7e85c4-636b-4d1b-99ce-9170332ceda7")    

s3_resource.Bucket("iris-trained7e85c4-636b-4d1b-99ce-9170332ceda7").delete()

[{'Key': '/data/training/iris.csv', 'VersionId': 'null'}, {'Key': '/model/iris-randomforest.pkl', 'VersionId': 'null'}, {'Key': 'data/training/iris.csv', 'VersionId': 'null'}, {'Key': 'model/iris-randomforest.pkl', 'VersionId': 'null'}, {'Key': 'train.py', 'VersionId': 'null'}]


{'ResponseMetadata': {'RequestId': 'DEF78351765EF038',
  'HostId': 'oW7euGnTPHmGMLLQZQKcda30Wudf+u0QxhsPhDd/mInTmjXRLFuwt0uCTOtZ//FePo3CxMZIwYY=',
  'HTTPStatusCode': 204,
  'HTTPHeaders': {'x-amz-id-2': 'oW7euGnTPHmGMLLQZQKcda30Wudf+u0QxhsPhDd/mInTmjXRLFuwt0uCTOtZ//FePo3CxMZIwYY=',
   'x-amz-request-id': 'DEF78351765EF038',
   'date': 'Sat, 02 Nov 2019 16:35:24 GMT',
   'server': 'AmazonS3'},
  'RetryAttempts': 0}}

[Go back to top](#top)
## Execute Code<a id=Execute_Code></a>

## EC2 Instance<a id=Execute_Code_EC2></a>

### Create EC2 Instance<a id=Execute_Code_EC2_Create></a>
https://blog.ipswitch.com/how-to-create-an-ec2-instance-with-python

You need to bring your own:
* Security Group
* pem key

We will be building the following EC2
* MiniConda - ami-062c42cbecc1d5ec0
* t2.medium

I built my own security group and granted ssh access.

In [143]:
import uuid
def create_ec2_name(bucket_prefix):
    # The generated bucket name must be between 3 and 63 chars long
    return ''.join([bucket_prefix, str(uuid.uuid4())])

Use a bash script to create the S3 Mount, go [here](#Execute_Code_EC2_Mount) to see details

In [173]:
import os
from dotenv import load_dotenv
load_dotenv(dotenv_path="../.env")

user_data = [
    "#cloud-boothook",
    "#!/bin/bash",
    "yum update -q -y",
    "yum install automake fuse fuse-devel gcc-c++ git libcurl-devel libxml2-devel make openssl-devel -q -y",
    "git clone https://github.com/s3fs-fuse/s3fs-fuse.git /tmp/s3fs-fuse",
    "cd /tmp/s3fs-fuse",
    "./autogen.sh",
    "./configure --prefix=/usr --with-openssl",
    "make",
    "make install",
    'echo "' + os.getenv("AWS_ACCESS_ID") + ':' + os.getenv("AWS_SECRET_ACCESS_KEY") + '" > /tmp/passwd-s3fs',
     "mv -f /tmp/passwd-s3fs /etc",
     "chmod 640 /etc/passwd-s3fs",
     "mkdir -p /mys3bucket",
     "chown ec2-user:ec2-user /mys3bucket",
     "s3fs iris-trainc0e3588c-d9bb-4699-821c-1883670ace42 -o use_cache=/tmp -o uid=500 -o mp_umask=002 -o multireq_max=5 /mys3bucket",
]
user_data = "\n".join(user_data)

In [172]:
import boto3
ec2 = boto3.resource('ec2')

# create a new EC2 instance
ec2_name = create_ec2_name("iris-train")
security_group = ['sg-0d24aec64507df8b5']
pem_key = "ballenger-smu"

instances = ec2.create_instances(
    TagSpecifications=[
        {
            'ResourceType': 'instance',
            'Tags': [
                {
                    'Key': 'Name',
                    'Value': ec2_name
                },
            ]
        }
    ],
    ImageId='ami-062c42cbecc1d5ec0',
    MinCount=1,
    MaxCount=1,
    InstanceType='t2.medium',
    KeyName=pem_key,
    SecurityGroupIds=security_group, #Bring your own!
    UserData=user_data
)

instances

[ec2.Instance(id='i-0be16185cfbd6f307')]

### Mount S3 onto EC2 Instance<a id=Execute_Code_EC2_Mount></a>
https://cloudkul.com/blog/mounting-s3-bucket-linux-ec2-instance/

Using existing EC2 in AWS
Need to leverage API to create EC2 and mount determining setup

**Required setup on EC2**
```
sudo yum update
sudo yum install automake fuse fuse-devel gcc-c++ git libcurl-devel libxml2-devel make openssl-devel
git clone https://github.com/s3fs-fuse/s3fs-fuse.git
cd s3fs-fuse
./autogen.sh
./configure --prefix=/usr --with-openssl
make
sudo make install
```

You must create an IAM role for S3 Mounting, for sake of simplicity, i'm using my Admin IAM Access
```
sudo touch /etc/passwd-s3fs
sudo vim /etc/passwd-s3fs
```
Provide `Your_accesskey:Your_secretkey` inside the file
```
sudo chmod 640 /etc/passwd-s3fs
```

Let's mount it!, replace iris-trainc0e3588c-d9bb-4699-821c-1883670ace42 with your bucket name
uid=500 is ec2-user account
```
sudo mkdir /mys3bucket
sudo chown ec2-user:ec2-user /mys3bucket
s3fs iris-trainc0e3588c-d9bb-4699-821c-1883670ace42 -o use_cache=/tmp -o allow_other -o uid=500 -o mp_umask=002 -o multireq_max=5 /mys3bucket
```

Validate
```
df -Th
```

Mount at reboot
```
vi /etc/rc.local
/usr/bin/s3fs iris-trainc0e3588c-d9bb-4699-821c-1883670ace42 -o use_cache=/tmp -o allow_other -o uid=500 -o mp_umask=002 -o multireq_max=5 /mys3bucket
```



Getting Conda to work

```
sudo chown -R ec2-user:ec2-user /opt/conda
conda update -n base -c defaults conda
conda env update -n base --file requirements.yml
```